In [1]:
from prevo.viewers import TkWindow, TkViewer
from prevo.viewers import CvWindow, CvViewer
from prevo.viewers import MplWindow, MplViewer

from prevo.misc import PeriodicSensor
from threading import Thread, Event
import oclock
import numpy as np
%matplotlib tk

# 1) MISC tools

## Dummy camera sensor

The section below is just to define a dummy class that mimicks a camera sending images on a queue.

In [2]:
class LapseCamera(PeriodicSensor):
    """Mock time-lapse camera returning white-noise images periodically"""
    
    name = 'Mock Lapse Camera'
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num = 0
    
    def _read(self):
        """Return image in a dict (see explanation below)"""
        img = np.random.randint(256, size=(480, 640), dtype='uint8')
        data = {'image': img, 'num': self.num}
        self.num += 1
        return data

In [3]:
camera1 = LapseCamera(interval=0.04)
camera1.start()

camera2 = LapseCamera(interval=2)
camera2.start()

## External command to stop display

In [4]:
def stop_display(after=10):
    """By default, send stop request after 10 seconds"""
    stop_event = Event()
    
    def _stop_display():
        timer = oclock.Timer()
        while timer.elapsed_time < 10:
            timer.checkpt()
        print('STOP REQUESTED')
        stop_event.set()
        
    Thread(target=_stop_display).start()
    return stop_event

# 2) Viewers operation

## Tkinter

In [5]:
win1 = TkWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = TkWindow(camera2.queue, name='Camera 2', show_num=True)

viewer = TkViewer(
    windows=(win1, win2),
    external_stop=stop_display(after=10),
)
viewer.start()

STOP REQUESTED


## OpenCV

In [6]:
win1 = CvWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = CvWindow(camera2.queue, name='Camera 2', show_num=True)

viewer = CvViewer(
    windows=(win1, win2),
    external_stop=stop_display(after=10),
)
viewer.start()

STOP REQUESTED


## Matplotlib

In [7]:
win1 = MplWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = MplWindow(camera2.queue, name='Camera 2', show_num=True)

viewer = MplViewer(
    windows=(win1, win2),
    blit=False,
    external_stop=stop_display(after=10),

)
viewer.start()

invalid command name "6434644736_on_timer"
    while executing
"6434644736_on_timer"
    ("after" script)


STOP REQUESTED
